# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [159]:
# Use this cell to write your code for Task 1
import pandas as pd
import numpy as np
df = pd.read_csv('house_sales.csv')
missing_city = (df['city']=='--').sum()
print(missing_city)

73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [160]:
clean_data.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   house_id       1500 non-null   category      
 1   city           1500 non-null   category      
 2   sale_price     1500 non-null   int64         
 3   sale_date      1500 non-null   datetime64[ns]
 4   months_listed  1500 non-null   float64       
 5   bedrooms       1500 non-null   int64         
 6   house_type     1500 non-null   category      
 7   area           1500 non-null   float64       
dtypes: category(3), datetime64[ns](1), float64(2), int64(2)
memory usage: 108.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   house_id       1500 non-null   int64  
 1   city           1500 non-null   object 
 2   sale_price   

In [161]:
# Use this cell to write your code for Task 2
import pandas as pd
import numpy as np
clean_data = pd.read_csv('house_sales.csv')

#Update area Column
clean_data['area'] = clean_data['area'].str.replace('sq.m.','')
clean_data['area'] = pd.to_numeric(clean_data['area'], errors='coerce')
# Replace missing with mean, rounded to one decimal place.
fill_value_area = round(clean_data['area'].mean(), 1)
clean_data['area'].fillna(fill_value_area, inplace=True)
clean_data['area'] = clean_data['area'].round(1).astype('float64')

#updat sale price column
clean_data.dropna(subset=['sale_price'], inplace=True)
clean_data['sale_price'] = clean_data['sale_price'].astype('int64')

#Update House ID column
clean_data['house_id'] = clean_data['house_id'].astype('category')

#Clean City Column
valid_cities = ['Silvertown', 'Riverford', 'Teasdale', 'Poppleton']
clean_data['city'] = clean_data['city'].apply(lambda x: x if x in valid_cities else 'Unknown')
clean_data['city'] = clean_data['city'].astype('category')

# Clean Sales Date column
clean_data['sale_date'] = pd.to_datetime(clean_data['sale_date'], errors='coerce')
clean_data['sale_date'].fillna(pd.to_datetime('2023-01-01'), inplace=True)

# Clean Month Listed column
fill_value_months = round(clean_data['months_listed'].mean(), 1)
clean_data['months_listed'].fillna(fill_value_months, inplace=True)
clean_data['months_listed'] = clean_data['months_listed'].round(1).astype('float64')

# Clean bedrooms column
fill_value_bedrooms = round(clean_data['bedrooms'].mean())
clean_data['bedrooms'].replace({np.nan: fill_value_bedrooms}, inplace=True)
clean_data['bedrooms'] = clean_data['bedrooms'].astype('int64')

# Clean House type

type_mapping = {
    'Det.': 'Detached',
    'Semi': 'Semi-detached',
    'Terr.': 'Terraced'
    
}


clean_data['house_type'].replace(type_mapping, inplace=True)


mode_house_type = clean_data['house_type'].mode()[0]
clean_data['house_type'].fillna(mode_house_type, inplace=True)


clean_data['house_type'] = pd.Categorical(
    clean_data['house_type'],
    categories=["Terraced", "Semi-detached", "Detached"],
    ordered=True
)

clean_data.head()

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [162]:
# Use this cell to write your code for Task 3
df_3 = pd.read_csv('house_sales.csv')
price_by_rooms = df_3.groupby('bedrooms')['sale_price'].agg(['mean','var'])
price_by_rooms.columns = ['avg_price','var_price']
price_by_rooms = price_by_rooms.round(1)
price_by_rooms = price_by_rooms.reset_index()
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [163]:
# Use this cell to write your code for Task 4
import pandas as pd
from sklearn.linear_model import LinearRegression

# --- Data Loading ---
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('validation.csv')

# --- 1. Separate Target Variable ---
y_train = df_train['sale_price']
X_train = df_train.drop('sale_price', axis=1)

# --- 2. Feature Engineering: Handle sale_date (The Fix) ---

def engineer_date_features(df):
    # Convert to datetime object first (Errors='coerce' handles invalid dates)
    df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
    
    # Extract numerical features
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    
    # Extract categorical feature (Day of Week)
    df['sale_day_of_week'] = df['sale_date'].dt.day_name()
    
    # DROP the original high-cardinality 'sale_date' column
    df = df.drop('sale_date', axis=1)
    return df

X_train_processed = engineer_date_features(X_train)
X_test_processed = engineer_date_features(df_test.copy()) # Use a copy of df_test

# --- 3. Handle Remaining Categorical Variables ---
# Remaining categorical columns: 'city', 'house_type', and the new 'sale_day_of_week'
categorical_cols = ['city', 'house_type', 'sale_day_of_week']

# Apply One-Hot Encoding to both training and test sets
X_train_encoded = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
X_test_encoded = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

# Align columns to ensure both dataframes have the exact same features
X_train_final, X_test_final = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Drop 'house_id' from the training set and remove any columns unique to the test set if alignment was outer/right
X_train_final = X_train_final.drop('house_id', axis=1)

# Prepare Test Set: Save house_id and drop it from features
house_ids = X_test_final['house_id']
X_test_final = X_test_final.drop('house_id', axis=1)
# Handle potential NaN columns created during alignment (if test set was missing features)
X_test_final = X_test_final.fillna(0) 
# Note: X_train_final will have NaN for columns missing in test, but since we align (join='left'),
# only X_test_final will have NaN if train had extra columns, which we fill with 0.

# --- 4. Model Fitting and Prediction ---
lr = LinearRegression()
lr.fit(X_train_final, y_train)

y_pred = lr.predict(X_test_final)

# --- 5. Final Output Creation ---
base_result = pd.DataFrame({
    'house_id': house_ids,
    'price': y_pred
})


print("Successfully created 'base_result' with reduced feature set.")

Successfully created 'base_result' with reduced feature set.


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [164]:
# Use this cell to write your code for Task 5
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# --- Data Loading ---
df_train_com = pd.read_csv('train.csv')
df_test_com = pd.read_csv('validation.csv')

# --- 1. Separate Target Variable ---
y_train_com = df_train_com['sale_price']
X_train_com = df_train_com.drop('sale_price', axis=1)

# --- 2. Feature Engineering: Handle sale_date (The Fix) ---

def engineer_date_features(df):
    # Convert to datetime object first (Errors='coerce' handles invalid dates)
    df['sale_date'] = pd.to_datetime(df['sale_date'], errors='coerce')
    
    # Extract numerical features
    df['sale_year'] = df['sale_date'].dt.year
    df['sale_month'] = df['sale_date'].dt.month
    
    # Extract categorical feature (Day of Week)
    df['sale_day_of_week'] = df['sale_date'].dt.day_name()
    
    # DROP the original high-cardinality 'sale_date' column
    df = df.drop('sale_date', axis=1)
    return df

X_train_processed = engineer_date_features(X_train_com)
X_test_processed = engineer_date_features(df_test_com.copy()) # Use a copy of df_test

# --- 3. Handle Remaining Categorical Variables ---
# Remaining categorical columns: 'city', 'house_type', and the new 'sale_day_of_week'
categorical_cols = ['city', 'house_type', 'sale_day_of_week']

# Apply One-Hot Encoding to both training and test sets
X_train_encoded_com = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
X_test_encoded_com = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

# Align columns to ensure both dataframes have the exact same features
X_train_final_com, X_test_final_com = X_train_encoded_com.align(X_test_encoded_com, join='left', axis=1, fill_value=0)

# Drop 'house_id' from the training set and remove any columns unique to the test set if alignment was outer/right
X_train_final_com = X_train_final_com.drop('house_id', axis=1)

# Prepare Test Set: Save house_id and drop it from features
house_ids_com = X_test_final_com['house_id']
X_test_final_com = X_test_final_com.drop('house_id', axis=1)
# Handle potential NaN columns created during alignment (if test set was missing features)
X_test_final_com = X_test_final_com.fillna(0) 
# Note: X_train_final will have NaN for columns missing in test, but since we align (join='left'),
# only X_test_final will have NaN if train had extra columns, which we fill with 0.

# --- 4. Model Fitting and Prediction ---
rf = RandomForestRegressor(n_estimators = 39, max_depth = 7, random_state=9)
rf.fit(X_train_final_com, y_train)

y_pred_com = rf.predict(X_test_final_com)

# --- 5. Final Output Creation ---
compare_result = pd.DataFrame({
    'house_id': house_ids_com,
    'price': y_pred_com
})


print("Successfully created 'compare_result' with reduced feature set.")


Successfully created 'compare_result' with reduced feature set.


In [165]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
param_dist = {'n_estimators': np.arange(1,101,1),
          'max_depth':np.arange(1,11,1)}
cv = RandomizedSearchCV(rf,param_distributions=param_dist,cv=5, random_state=9)
cv.fit(X_train_final,y_train)
# Print the tuned parameters and score
print("Tuned Random Forest Regressor Parameters: {}".format(cv.best_params_))
print("Tuned Random Forest Regressor Best Accuracy Score: {}".format(cv.best_score_))


Tuned Random Forest Regressor Parameters: {'n_estimators': 39, 'max_depth': 7}
Tuned Random Forest Regressor Best Accuracy Score: 0.9838515043684236


In [166]:
from sklearn.model_selection import train_test_split
df_split = pd.read_csv('house_sales.csv')
X = df_split.drop("sale_price",axis=1)
y = df_split['sale_price']
X_train_check, X_test_check, y_train_check, y_test_check = train_test_split(X,y, test_size = 0.2,random_state=9)
from sklearn.metrics import mean_squared_error 

mse = mean_squared_error(y_test_check,y_pred)
rmse = mse**(1/2)
rmse

164657.2079166194